In [218]:
# Start by importing numpy
import numpy as np

# Load the data for the problem
my_data = np.load('A.npz')
bag_capacity = int(my_data['capacity'])
n_items = int(my_data['n_items'])
item_values = my_data['item_values']
item_weights = my_data['item_weights']

In [219]:
# Evaluate the starting population
def calc_fitness(pop):
    fitness = pop @ item_values
    return fitness

# Choose the parents
n_mating = 4

def select_parents(pop, pop_fitness):
    idx = np.argsort(pop_fitness)
    idx = idx[::-1]
    parents = pop[idx]
    
    # Select best of pop
    parents = parents[:n_mating]
    return (parents,calc_fitness(parents))

In [220]:
# Recombination
n_offspring = 3
crossover_idx = 3

def crossover(parents):
    offspring = np.empty((n_offspring, n_items))
    
    for k in range(n_offspring):        
        # Index of the first parent to mate.
        p1_idx = k%n_mating

        # Index of the second parent to mate.
        p2_idx = (k+1)%n_mating
        
        offspring[k, :crossover_idx] = parents[p1_idx, :crossover_idx]
        offspring[k, crossover_idx:] = parents[p2_idx, crossover_idx:]
        
    return (offspring, calc_fitness(offspring))

In [221]:
# Mutation
mutation_prob = 0.5
def mutation(offspring):
    random_mutator = np.random.uniform(0.0, 1.0, (n_offspring,))
    
    idx = random_mutator > mutation_prob
    number_of_nonzeros = np.count_nonzero(idx)
    mutated_offspring = offspring.copy()
    mutated_offspring[idx] += np.random.uniform(-0.5,0.5,(number_of_nonzeros,n_items))
    return (mutated_offspring, calc_fitness(mutated_offspring))

In [222]:
def calc_weight(pop):
    weights = pop @ item_weights
    return weights

In [223]:
max_weight = bag_capacity
# eliminate members that don't meet weight constraint
def weight_constraint(pop):
    idx = (calc_weight(pop) <= max_weight)
    return pop[idx]

In [224]:
# rank members of big population and cut down to pop_size
def environmental_selection(pop, pop_fitness):
    idx = np.argsort(pop_fitness)
    idx = idx[::-1]
    pop = pop[idx]
    
    # Select best of pop
    pop = pop[0:pop_size]

    return (pop,calc_fitness(pop))

In [225]:
def generate_initial_pop():
    # Represent the problem
    pop_size = 10
    dofs_in_pop = (pop_size,n_items)
    

    # starting population; each row is a member, each col an item
    mu = 0.0 # mean
    sigma = 0.3 # st. dev, spread
    pop = np.absolute(np.round(np.random.normal(loc = mu, scale = sigma, size = dofs_in_pop))) 
    pop = weight_constraint(pop)
    
    # make sure all the starting pop meets weight constraint
    while pop[:,1].size < pop_size:
        dofs_in_pop = (pop_size-pop[:,1].size,n_items)
        new_members = np.absolute(np.round(np.random.normal(loc = mu, scale = sigma, size = dofs_in_pop)))
        pop = np.vstack((pop,new_members))
        pop = weight_constraint(pop)
    
    return pop

In [228]:
# algorithm to find optimal solution
pop = generate_initial_pop()

# iterate to approach optimal solution
n_generations = 500000
    
for i in range(n_generations):
    parents,_ = select_parents(pop,calc_fitness(pop))
    offspring,_ = crossover(parents)
    mutated_offspring,_ = mutation(offspring)
    pop = np.vstack((pop, mutated_offspring))
    pop = weight_constraint(pop)
    pop,_ = environmental_selection(pop, calc_fitness(pop))
    
calc_fitness(pop)

array([591073.8106834 , 591070.68631213, 591070.68631213, 591069.5356359 ,
       591067.53628077, 591064.64107357, 591062.79448519, 591062.37888572,
       591062.37888572, 591062.37888572])

In [229]:
calc_weight(pop)

array([ 9.03466008, 17.57998845, 17.57998845, 19.23512888, 18.87345458,
        0.7669482 , 15.3040466 , 18.16360676, 18.16360676, 18.16360676])